In [1]:
# Generic python libraries
import os
import math
import random
from PIL import Image
from random import sample

# PyTorch and numpy related
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split

In [2]:
  def image_stats(image_list):
    widths = []
    heights = []
    for img in image_list:
      widths.append(img.width)
      heights.append(img.height)        
    print(f"Max: {max(widths)}, {max(heights)}")
    print(f"Min: {min(widths)}, {min(heights)}")
    print(f"Mean: {np.average(widths)}, {np.average(heights)}")

In [3]:
def get_padding(image):    
    w, h = image.size
    max_wh = np.max([w, h])
    h_padding = (max_wh - w) / 2
    v_padding = (max_wh - h) / 2
    l_pad = h_padding if h_padding % 1 == 0 else h_padding+0.5
    t_pad = v_padding if v_padding % 1 == 0 else v_padding+0.5
    r_pad = h_padding if h_padding % 1 == 0 else h_padding-0.5
    b_pad = v_padding if v_padding % 1 == 0 else v_padding-0.5
    padding = (int(l_pad), int(t_pad), int(r_pad), int(b_pad))
    return padding

class CustomPad(object):
    def __init__(self, fill=0, padding_mode='constant'):
        assert padding_mode in ['constant', 'edge', 'reflect', 'symmetric']

        self.fill = fill
        self.padding_mode = padding_mode
        
    def __call__(self, img):
        """
        Args:
            img (PIL Image): Image to be padded.

        Returns:
            PIL Image: Padded image.
        """
        padding = torchvision.transforms.Pad(get_padding(img),self.fill, self.padding_mode)
        return padding(img)

In [4]:
class ObjectDetectionTrainDataset(Dataset):
  
  def __init__(self):
        
    self.boxes = []
    self.raw_images = []
    self.processed_images = []
    base_path = "/home/humberto/src/Univ/DATASETS/orand-car-with-bbs/training"
    current_annotations = os.listdir(f"{base_path}/annotations")
    
    preprocess_pipeline = transforms.Compose([
      CustomPad(),
      transforms.Resize(600)
    ])
    
    
    for annotation in current_annotations:
        # Parsing File names structure
        annotation_file = f"{base_path}/annotations/{annotation}"
        image_file = base_path +"/images/"+annotation.split(".")[0]+".png"
        
        # Opening Both annotations and image file
        annotation_file = open(annotation_file, "r")
        annotation_lines = annotation_file.readlines()
        image = Image.open(image_file).convert("RGB")
        # Storing tensor version of the image and raw_image as well
        self.raw_images.append(image)
        self.processed_images.append(image)
        
        # Building the required structure for boxes and classes
        coords = []
        classes = []
        for line in annotation_lines:
            digit_class, boundaries = line.split(":")
            x1, y1, w, h = [ int(x) for x in boundaries.strip().split(",") ]
            coords.append([x1,y1,x1+w,y1+h])
            classes.append(int(digit_class))
        
        self.boxes.append({
            "labels": torch.tensor(classes),
            "boxes": torch.tensor(coords)
        })
        annotation_file.close()
      
  def __len__(self):
    return len(self.processed_images)

  def __getitem__(self, idx):
    return self.processed_images[idx], self.boxes[idx]



In [15]:
da = Image.open("/home/humberto/src/Univ/DATASETS/orand-car-with-bbs/training/images/anv_5556813_4177.png").convert("RGB")
print(da.format)
for r in np.asarray(da):
    print(r)


None
[[255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255

In [5]:
t_dataset = ObjectDetectionTrainDataset()

/home/humberto/src/Univ/DATASETS/orand-car-with-bbs/training/images/anv_5556813_4177.png 80 266


KeyboardInterrupt: Interrupted by user

In [ ]:
train_dataset_size = int(0.8 * len(t_dataset))
val_dataset_size = int(len(t_dataset) - train_dataset_size)

# Splitting train data into validation and train
train_dataset, val_dataset = random_split(t_dataset, [train_dataset_size, val_dataset_size])

In [ ]:
for img in val_dataset:
    img[0].show()
    x = input()

In [ ]:
print(f"Train dataset size: {len(train_dataset)} -- Val Dataset size: {len(val_dataset)}")

In [ ]:
def collater(batch):
    img_data = [x[0] for x in batch]
    box_data = [x[1] for x in batch]
    return torch.stack(img_data), box_data

In [ ]:
def train(data, network, optimizer, device, lamb=0.6):
  network.train()
  loss = 0
  i = 0
  for input, target in data:
    input = input.to(device).float()
    for t in target:
        t["labels"] = t["labels"].to(device)
        t["boxes"] = t["boxes"].to(device)
    output = network(input, target)
    current_loss = (lamb * output["classification"]) + ((1-lamb) * output["bbox_regression"])
    loss += current_loss.item()
    current_loss.backward()
    optimizer.step()
    break
    if i % 30 == 0:
        print(f"Batch: {i}")
    i+=1
  return loss

def validation(data, network, device):
  network.eval()
  acc_c = 0
  with torch.no_grad():
    for input, target in data:
      input = input.to(device).float()
      for t in target:
        t["labels"] = t["labels"].to(device)
        t["boxes"] = t["boxes"].to(device)
      output = network(input)
      print(output)
    return 0

#run epochs -> 
def run_epochs(network, train_dataset, val_dataset, optimizer, epochs=5, batch_size=24, device='cuda', collate=collater):
  network.to(device)
  train_data = DataLoader(train_dataset,batch_size=batch_size,shuffle=True,collate_fn=collate)
  val_data = DataLoader(val_dataset,batch_size=batch_size,shuffle=True,collate_fn=collate)
  train_loss, train_acc = [], []
  val_loss, val_acc = [], []
  for e in range(1,epochs+1):
    t_loss = train(train_data, network, optimizer, device)
    v_acc = validation(val_data, network, device)

    train_loss.append(t_loss/len(train_dataset)*batch_size)
    val_acc.append(v_acc/len(val_dataset)*100)

#     print(f"Epoch: {e} \n Train | Acc: {train_acc[-1]:.3f}% | loss: {train_loss[-1]:.3f} \n Val   | Acc: {val_acc[-1]:.3f}% | loss: {val_loss[-1]:.3f}")
    print(f"Epoch: {e} \n Train | loss: {train_loss[-1]:.3f}")

    #   generate_comparison_charts(train_loss,val_loss,train_acc,val_acc)

In [ ]:
LR = 1e-5
model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=False,num_classes=10)
optimizer = optim.AdamW(model.parameters(),lr=LR)
run_epochs(model,train_dataset,val_dataset,optimizer)